Code for UAV lab project

In [ ]:
import pandas as pd
import numpy as np
from json import loads

Methods used:

**download (if not already downloaded), then upload the txt. file colab to run code**

 https://drive.google.com/file/d/1CnCDpMl9y4iup3bMu8QiCI4asl5WPkY_/view?usp=sharing

In [ ]:
def open_lidar(filename, verbose=False):
    """Method for opening LiDAR text files and handling possible line errors
    """
    # open file and read in each line
    with open(filename, "r") as f:
        lines = f.readlines()

    # iterate through lines to cast to lists of floats
    new_lines = list()
    for line in lines:

        # in case file is corrupt 
        try:
            new_lines.append([float(val) for val in line.split()])

        except Exception as e:
            if verbose:  # printing only if verbose, ignore otherwise
                print(line)

    # convert nested list to pandas dataframe
    new_lines = pd.DataFrame(new_lines)
    new_lines = new_lines.rename(columns={0: "x", 1: "y", 2: "z"})

    return new_lines


Importing Data

In [ ]:
# open and import sample dataset
lidar_df = open_lidar("N092E301.txt", True)            # make sure your uploaded file matches in name and format

In [ ]:
# save out a small number of data points for testing
with open("N092E301_small.json", "w") as fh:
  fh.write(lidar_df[:128].to_json())
  fh.close()

In [ ]:
#calculated the max values, the min, the scale(max-min) and the median value 
max = lidar_df.max() 
min = lidar_df.min()

# organized the data into a list to be converted into a dataframe  
lidar_stats = pd.DataFrame(
    [min, max, max - min],
    index=["min", "max", "scale"]
)

In [ ]:
# export stats dataframe so the full dataset doesn't have to be loaded everytime this analysis is performed
with open("N092E301_stats.json", "w") as fh:
  fh.write(lidar_stats.to_json())
  fh.close()

In [ ]:
# import stats dataframe
with open("N092E301_stats.json", "r") as fh:
  lidar_stats = pd.DataFrame(loads(fh.read()))
  fh.close()

Create Grid DataFrame

In [ ]:
side = 2048
increment = side / lidar_stats.loc["scale"]
point_grid = pd.DataFrame(
    index=[y for y in np.arange(lidar_stats["y"]["min"], lidar_stats["y"]["max"], increment["y"])],
    columns=[x for x in np.arange(lidar_stats["x"]["min"], lidar_stats["x"]["max"], increment["x"])],
)

In [ ]:
print(lidar_stats)
print(increment)
print(point_grid)

                x           y        z
min    5280000.00  3902958.42   327.73
max    5284999.99  3905000.00  5633.83
scale     4999.99     2041.58  5306.10
x    0.409601
y    1.003145
z    0.385971
Name: scale, dtype: float64
             5.280000e+06 5.280000e+06  ... 5.284999e+06 5.285000e+06
3.902958e+06          NaN          NaN  ...          NaN          NaN
3.902959e+06          NaN          NaN  ...          NaN          NaN
3.902960e+06          NaN          NaN  ...          NaN          NaN
3.902961e+06          NaN          NaN  ...          NaN          NaN
3.902962e+06          NaN          NaN  ...          NaN          NaN
...                   ...          ...  ...          ...          ...
3.904996e+06          NaN          NaN  ...          NaN          NaN
3.904997e+06          NaN          NaN  ...          NaN          NaN
3.904998e+06          NaN          NaN  ...          NaN          NaN
3.904999e+06          NaN          NaN  ...          NaN          NaN
3.90

Grid LiDAR Data Points

In [ ]:
#This function will help form a LiDAR grid with a proper set of consistent points
def calc_elevation(x, y, nearby_data_points):
  """Helper method for calculating the elevation based off nearby points.
  """

  # if not nearby_data_points:
  #   return None

  distances = [((val[0] - x)**2 + (val[1] - y)**2)**0.5 for val in nearby_data_points]
  total_distance = len(distances)
  return sum([val[2] * (distances[index]/total_distance) for index, val in enumerate(nearby_data_points)])


gridded_df = pd.DataFrame(columns=np.arange(lidar_stats["x"]["min"], lidar_stats["x"]["max"], increment["x"]))

for x in np.arange(lidar_stats["x"]["min"], lidar_stats["x"]["max"], increment["x"]):
  for y in np.arange(lidar_stats["y"]["min"], lidar_stats["y"]["max"], increment["y"]):
    values = lidar_df[
                  
                      (lidar_df["x"] >= x) & \
                      (lidar_df["x"] <= (x + increment["x"])) & \
                      (lidar_df["y"] >= y) & \
                      (lidar_df["y"] <= (y + increment["y"]))
                      ]
    delete_indexes = values.index
    lidar_df.drop(labels=delete_indexes, axis=0, inplace=True)

    
    gridded_df[x][y] = calc_elevation(x, y, values.values)

    # if the values are less than both the y and x value delete them from the array (reduces computation time A LOT!)